## 数据预处理

In [ ]:
# https://huggingface.co/datasets/Maciel/FinCUGE-Instruction
def custom_FinCUGE(from_dir, to_dir):
    from datasets import load_dataset, concatenate_datasets
    import os
    dataset = load_dataset(from_dir)  
    for split_name in dataset:
        dataset[split_name] = dataset[split_name].map(lambda example: {"split": split_name})
    combined_data = concatenate_datasets([dataset[split_name] for split_name in dataset])
    os.makedirs(to_dir, exist_ok=True)
    df = combined_data.to_pandas()
    df.to_json(os.path.join(to_dir,"FinCUGE.jsonl"), orient='records',force_ascii=False, lines=True)

def extract_finna_in_FinCUGE_as_sharegpt(from_dir, to_dir):
    import pandas as pd  
    import os
    df = pd.read_json(f"{from_dir}/FinCUGE.jsonl", lines=True)
    df['messages'] = df.apply(lambda row: [
                                dict(role='system',content=row['instruction']),
                                dict(role='user',content=row['input']),
                                dict(role='assistant',content=row['output'])
                            ], axis=1)
    df[(df['split'] == 'train') & (df['task'] == 'FINNA')][['messages']].to_json(os.path.join(to_dir,"FinCUGE_FINNA_train.jsonl"), orient='records',force_ascii=False, lines=True)